In [4]:
import pandas as pd
import nltk
import re
import numpy as np
import time 
import pickle
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

#for RNN
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
#import pre-processed data from pickle 
df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/sample_results_df_09.pickle")
df_09

,pre_processed_sent,male_count,female_count,apicall_fail,sentences,article_id,year,col_type
0,"[break, silence, surround, break, say, band, l...",3,0,0,Liam Gallagher has broken the silence surround...,5048,2009,0
1,"[however, interview, say, longer]",2,0,0,"However, in an interview with The Times Liam G...",5048,2009,0
9,"[leave, band, follow, bust, say, simply, could...",4,0,0,Noel Gallagher left the Manchester band follow...,5048,2009,0
10,"[launch, clothing, line, earlier, year, admit,...",3,0,0,"""Liam launched his clothing line Pretty Green ...",5048,2009,0
13,"[people, able, buy, record]",1,0,0,"""People will be able to buy his records.",5048,2009,0
...,...,...,...,...,...,...,...,...
9706,"[organisers, say, extend, programme, live, tou...",2,0,0,Organisers of the X Factor have said they've e...,1043733,2009,0
9707,"[vote, judge, week, seven, competition]",1,0,0,Jedward were voted off by the X Factor judges ...,1043733,2009,0
9708,"[two, month, tour, begin, see, extra, date, add]",3,0,0,"The two month tour, which begins in Liverpool ...",1043733,2009,0
9709,"[artists, confirm, tour, contestant]",2,1,0,Artists confirmed for the tour are contestants...,1043733,2009,0


**Define TFIDF Vectorizer**

In [3]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Word2Vec for Word Embedding**

In [3]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# Train a Word2Vec model on tokenized sentences
model_w2v = Word2Vec(df_09["pre_processed_sent"], size=100, window=5, min_count=1, workers=4)

# Convert the sentences into feature vectors using the Word2Vec model
X = np.array([np.mean([model_w2v.wv[word] for word in sentence], axis=0) for sentence in df_09["pre_processed_sent"]])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df_09["col_type"], test_size=0.2, random_state=42)


TypeError: __init__() got an unexpected keyword argument 'size'

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [7]:
#word2vec to dos
import gensim
from gensim.models import Word2Vec
file_w2vec= (r"/Users/yolandaferreirofranchi/Desktop/GoogleNews-vectors-negative300.bin") #yolanda's path 
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(file_w2vec, binary=True)

In [28]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils import check_array

def logistic_regression_year_wv(df, target_col, text_col):
    # Define the pipeline
    preprocessor = make_column_transformer(
        (make_pipeline(CountVectorizer(), TfidfTransformer()), text_col),
        remainder='passthrough'
    )
    clf = make_pipeline(preprocessor, FunctionTransformer(check_array), LogisticRegression())

    # Split the data into train and test sets
    X = df[text_col]
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the grid of hyperparameters to search over
    param_grid = {'logisticregression__C': [0.1, 1, 10]}

    # Create a GridsearchCV object
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = LogisticRegression(**grid_search.best_params_)
    best_clf.fit(X_train, y_train)
    print(f"Train accuracy: {best_clf.score(X_train, y_train)}")
    print(f"Test accuracy: {best_clf.score(X_test, y_test)}")
    return best_clf


In [24]:
def logistic_regression_year_wv(df, target_col, text_col):
    #start timer 
    start_time = time.time()

    # Train a Word2Vec model on the tokenized sentences
    model_w2v = Word2Vec(df[text_col], min_count=1, workers=4)

    # Transform the sentences into feature vectors using the Word2Vec model
    X = [np.mean([model_w2v.wv[token] for token in sentence], axis=0) for sentence in df[text_col]]

    # Extract the labels from the dataframe
    y = df[target_col]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga'], 
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:100]
    low_coef = sorted_coef[-100:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])


    #print probability results 
    feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    positive_probas = probas[:, 1]

    print(f"\nProbability of the Following Words Being Female:")
    feature_prob_dict = {}
    for feature, index in feature_indices.items():
        proba = (X_transformed[:, index].toarray() * positive_probas).mean()
        feature_prob_dict[feature] = proba
        print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #return df_probs #DF of probability for each word being female by year

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef

In [29]:
logistic_regression_year_wv(df_09, 'col_type', 'pre_processed_sent')

IndexError: tuple index out of range

In [30]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag' 'saga'], 
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:100]
    low_coef = sorted_coef[-100:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])


    #print probability results 
    feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    positive_probas = probas[:, 1]

    print(f"\nProbability of the Following Words Being Female:")
    feature_prob_dict = {}
    for feature, index in feature_indices.items():
        proba = (X_transformed[:, index].toarray() * positive_probas).mean()
        feature_prob_dict[feature] = proba
        print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #return df_probs #DF of probability for each word being female by year

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef

In [31]:
logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')

Accuracy: 0.66
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       418
           1       0.55      0.58      0.56       248

    accuracy                           0.66       666
   macro avg       0.64      0.65      0.64       666
weighted avg       0.67      0.66      0.67       666


Probability of the Following Words Being Female:
abandon: 0.00038120850269269033
abc: 0.0001413201190697638
abdomen: 7.339213954914968e-05
abiding: 5.2375749347855515e-05
ability: 7.871855922404075e-05
able: 0.0008132952952532201
aboard: 3.773704745159681e-05
abound: 4.120399451172335e-05
absence: 0.0002264022740987104
absolutely: 0.00035186146183954124
absoultely: 5.1772073732715285e-05
abuse: 0.0006958814132373457
abusive: 9.764637581888604e-05
academy: 9.623521697652816e-05
accept: 0.0003250303697333194
acceptance: 4.028711363185348e-05
accepting: 5.44636593844929e-05
access: 0.00020352834725587243
accessible: 4.33494228737

(       feature      coef
 0          say  2.771061
 1     newsbeat  1.650766
 2       really  1.560488
 3         time  1.445050
 4          add  1.425308
 ..         ...       ...
 95        song  0.656671
 96  understand  0.655653
 97      barely  0.653039
 98         mia  0.652188
 99        fair  0.652179
 
 [100 rows x 2 columns],
         feature      coef
 0         stoke -0.558318
 1          bank -0.562005
 2       virtual -0.567446
 3        either -0.573952
 4          east -0.574304
 ..          ...       ...
 95      country -1.203597
 96  forthcoming -1.237517
 97         hold -1.267933
 98         band -1.411951
 99      include -1.469173
 
 [100 rows x 2 columns])

**Coefficient Analysis** 

In [7]:
#call the function - with accuracy, classification, highest & lowest coeffs, df with word probs
df_coef_09 = logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')
df_coef_09

Accuracy: 0.66
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       418
           1       0.55      0.58      0.56       248

    accuracy                           0.66       666
   macro avg       0.64      0.65      0.64       666
weighted avg       0.67      0.66      0.67       666


Probability of the Following Words Being Female:
abandon: 0.00038120850269269033
abc: 0.0001413201190697638
abdomen: 7.339213954914968e-05
abiding: 5.2375749347855515e-05
ability: 7.871855922404075e-05
able: 0.0008132952952532201
aboard: 3.773704745159681e-05
abound: 4.120399451172335e-05
absence: 0.0002264022740987104
absolutely: 0.00035186146183954124
absoultely: 5.1772073732715285e-05
abuse: 0.0006958814132373457
abusive: 9.764637581888604e-05
academy: 9.623521697652816e-05
accept: 0.0003250303697333194
acceptance: 4.028711363185348e-05
accepting: 5.44636593844929e-05
access: 0.00020352834725587243
accessible: 4.33494228737

(       feature      coef
 0          say  2.771061
 1     newsbeat  1.650766
 2       really  1.560488
 3         time  1.445050
 4          add  1.425308
 ..         ...       ...
 95        song  0.656671
 96  understand  0.655653
 97      barely  0.653039
 98         mia  0.652188
 99        fair  0.652179
 
 [100 rows x 2 columns],
         feature      coef
 0         stoke -0.558318
 1          bank -0.562005
 2       virtual -0.567446
 3        either -0.573952
 4          east -0.574304
 ..          ...       ...
 95      country -1.203597
 96  forthcoming -1.237517
 97         hold -1.267933
 98         band -1.411951
 99      include -1.469173
 
 [100 rows x 2 columns])

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [9]:
#create DF of highest coef
highest_coef_09 = pd.DataFrame(df_coef_09[0]) 

#create DF of lowest lowest coef manipulation 
lowest_coef_09 = pd.DataFrame(df_coef_09[1]) 
lowest_coef_09 = lowest_coef_09.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
lowest_coef_09

,feature,coef
0,include,-1.469173
1,band,-1.411951
2,hold,-1.267933
3,forthcoming,-1.237517
4,country,-1.203597
...,...,...
95,east,-0.574304
96,either,-0.573952
97,virtual,-0.567446
98,bank,-0.562005


In [ ]:
#save DF as pickle file per year 
lowest_coef_09.to_pickle('RESULTS09_coeflow_sample.pickle')
highest_coef_09.to_pickle('RESULTS09_coeflhigh_sample.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Word Probability Analysis** 

#make sure to un-comment the last two lines of the functions

In [10]:
#call the function - with accuracy, classification, highest & lowest coeffs, df with word probs
df_probs_09 = logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')
df_probs_09

Accuracy: 0.66
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       418
           1       0.55      0.58      0.56       248

    accuracy                           0.66       666
   macro avg       0.64      0.65      0.64       666
weighted avg       0.67      0.66      0.67       666


Probability of the Following Words Being Female:
abandon: 0.00038120850269269033
abc: 0.0001413201190697638
abdomen: 7.339213954914968e-05
abiding: 5.2375749347855515e-05
ability: 7.871855922404075e-05
able: 0.0008132952952532201
aboard: 3.773704745159681e-05
abound: 4.120399451172335e-05
absence: 0.0002264022740987104
absolutely: 0.00035186146183954124
absoultely: 5.1772073732715285e-05
abuse: 0.0006958814132373457
abusive: 9.764637581888604e-05
academy: 9.623521697652816e-05
accept: 0.0003250303697333194
acceptance: 4.028711363185348e-05
accepting: 5.44636593844929e-05
access: 0.00020352834725587243
accessible: 4.33494228737

KeyboardInterrupt: 

In [ ]:
topwords_09 = df_probs_09["probability"].sort_values(ascending= False)

#top 100 words with highest probability of belonging to the female class 
topwords_09 = pd.DataFrame(topwords_09) 
topwords_09.to_pickle('RESULTS09_sample.pickle')
topwords_09.head(100)

**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

**Support Vector Machine**

In [19]:
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
def svm_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X).toarray()
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'C': [.001, .01, .1, 1, 10, 100],
        #'kernel': default('rbf')',
        'degree': [2,3,4],
        'gamma': ['scale', 'auto'],
        #'class_weight': [None],
        #'random_state': [42]
    }

    #the classifier 
    clf = SVC()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = SVC(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Year: {year}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #get important features (equivalent of coefficients in logreg)
    result_clf = permutation_importance(best_clf, X_test, y_test, n_repeats=10, random_state=42)
    importance_scores = result_clf.importances_mean
    importance_scores_df = pd.DataFrame(importance_scores, columns=['tbd'])

    for i in range(len(importance_scores)):
        print("Feature {}: Importance score = {:.3f}".format(i, importance_scores[i]))
    
    #print probability results 
    #feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    #probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    #positive_probas = probas[:, 1]

    #print(f"\nProbability of the Following Words Being Female:")
    #feature_prob_dict = {}
    #for feature, index in feature_indices.items():
        #proba = (X_transformed[:, index] * positive_probas).mean()
        #feature_prob_dict[feature] = proba
        #print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #return df_probs #DF of probability for each word being female by year

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return importance_scores_df

In [20]:
svm_year(df_09, 'col_type', 'pre_processed_sent') #takes more than 1 hour to run

Accuracy: 0.70
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.88      0.78       418
           1       0.66      0.38      0.48       248

    accuracy                           0.70       666
   macro avg       0.68      0.63      0.63       666
weighted avg       0.69      0.70      0.67       666



KeyboardInterrupt: 

In [ ]:
#save the SVM model 
import joblib
model, accuracy, class_report = svm_year(df_09, 'col_type', 'pre_processed_sent')

# Save the model and performance metrics
joblib.dump(model, 'model_svm.pkl')
with open('performance_metrics.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Classification Report:\n{class_report}")

**Neural Network** 

In [30]:
def RNN_lstm(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    # Preprocess text
    tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post', padding='post')
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)
    
    # Define model
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(10000, 32),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    
    # Evaluate model on test set
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

In [31]:
RNN_lstm(df_09, 'col_type', 'pre_processed_sent')

Epoch 1/10


2023-04-06 18:18:08.183364: W tensorflow/core/framework/op_kernel.cc:1757] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py

**Comparing the Models**

In [25]:
model_results = {' ': ['Logistic Regression', 'SVM', 'NN'],
           'Accuracy': [0.66, 0.70, 0],
           'Time (s)': [54.39, 3540, 0]}

# Create a pandas dataframe from the dictionary
df_mod_results = pd.DataFrame(model_results)

# Set the index of the dataframe to the Kernel column
df_mod_results.set_index(' ', inplace=True)

# Display the dataframe
print(df_mod_results)

                     Accuracy  Time (s)
                                       
Logistic Regression      0.66     54.39
SVM                      0.70   3540.00
NN                       0.00      0.00
